In this demo, we will use Gaussian mixture model to do background subtraction.

In [1]:
!pip install -q gdown

In [2]:
!gdown --id 1LMNA-pvknbEp2fiykYM4EWMmLmyYtMki -O fall_scenario8.zip
!unzip -q fall_scenario8.zip -d fall_scenario8_folder

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1LMNA-pvknbEp2fiykYM4EWMmLmyYtMki
From (redirected): https://drive.google.com/uc?id=1LMNA-pvknbEp2fiykYM4EWMmLmyYtMki&confirm=t&uuid=f6a8acd3-e42e-4923-a1ed-5fe35535eb4a
To: /content/fall_scenario8.zip
100% 73.0M/73.0M [00:00<00:00, 165MB/s]


We first download a video from the [multi-camera fall dataset](https://www.iro.umontreal.ca/~labimage/Dataset/?utm_source=chatgpt.com). The example used here is a simple scenario where a person walks into a room and then falls to the ground. Let's play the video to see what it looks like:

In [18]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import imageio
from IPython.display import HTML
from base64 import b64encode
from sklearn.model_selection import train_test_split

# --- Load image sequence ---
folder = "/content/fall_scenario8_folder/fall_scenario8_cam2/"
frame_paths = sorted(glob.glob(os.path.join(folder, "*.png")))

frames = []
for idx, path in enumerate(frame_paths):
    img = cv2.imread(path)
    if img is None:
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.asarray(img, dtype=np.float64)
    img /= 255.0
    frames.append(img)

frames = np.asarray(frames, dtype=np.float64)
print(f'frame: {frames.shape}')

# --- Train/Test split (no shuffle for temporal data) ---
train_frames, test_frames = train_test_split(frames, train_size=0.4, shuffle=False)
print(f'train_frames: {train_frames.shape}')
print(f'test_frames: {test_frames.shape}')

# Stack frames to compare two sets of frames:
# stacked_frames = [np.hstack((a, b)) for a, b in zip(train_frames, test_frames)]
stacked_frames = train_frames

# --- Convert to 8-bit and save as MP4 ---
os.makedirs("output", exist_ok=True)
video_path = "output/stacked_video.mp4"

# Convert float64 (0–1) → uint8 (0–255)
frames_uint8 = (np.clip(stacked_frames, 0, 1) * 255).astype(np.uint8)

# Save video using imageio
imageio.mimsave(video_path, frames_uint8, fps=10)

# --- Display the video inline in Colab ---
mp4 = open(video_path, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=640 controls>
    <source src="{data_url}" type="video/mp4">
</video>
""")

frame: (110, 480, 720, 3)
train_frames: (44, 480, 720, 3)
test_frames: (66, 480, 720, 3)


We create a class `GMM` that fit the Gaussian mixture model to the intensity of the image. Specifically, a Expectation–Maximization (EM) algorithm is adopted to estimate the parameters of multiple Gaussian distributions that model the data.

**Overview**

The Gaussian Mixture Model (GMM) is a probabilistic model that assumes the data
are generated from a mixture of several Gaussian (normal) distributions with unknown parameters. Each Gaussian component represents a cluster, and the algorithm aims to estimate the parameters of these Gaussians using the Expectation-Maximization (EM) algorithm.

**Expectation Step (E-step)**

During the Expectation step, the algorithm computes the responsibilities:
\begin{equation}
    \gamma_{nk} = \frac{\pi_k \, \mathcal{N}(x_n | \boldsymbol{\mu}_k, \Sigma_k)}
    {\sum_{j=1}^{K} \pi_j \, \mathcal{N}(x_n | \boldsymbol{\mu}_j, \Sigma_j)},
\end{equation}
where $\mathcal{N}(x | \boldsymbol{\mu}, \Sigma)$ is the multivariate normal density. Intuitively, $\gamma_{nk}$ represents how strongly data point $x_n$ is associated with Gaussian component $k$ given the current parameters.

**Maximization Step (M-step)**

In the Maximization step, the parameters are updated to maximize the expected
log-likelihood based on the current responsibilities:
\begin{align}
    N_k &= \sum_{n=1}^{N} \gamma_{nk}, \\
    \boldsymbol{\mu}_k &= \frac{1}{N_k} \sum_{n=1}^{N} \gamma_{nk} x_n, \\
    \Sigma_k &= \frac{1}{N_k} \sum_{n=1}^{N} \gamma_{nk} (x_n - \boldsymbol{\mu}_k)(x_n - \boldsymbol{\mu}_k)^{T}, \\
    \pi_k &= \frac{N_k}{N}.
\end{align}

These updates ensure that the parameters move toward values that maximize
the likelihood of the observed data under the mixture model.

**Fitting Procedure**

The `fit(X)` method alternates between the E-step and M-step until the maximum number of iterations is reached.

**Prediction**

The `predict(X)` method assigns each sample $x_n$ to the component $k$ with the highest posterior probability:
\begin{equation}
    \hat{z}_n = \arg\max_k \gamma_{nk}.
\end{equation}


In [11]:
class GMM(object):

    weights = None
    means = None
    covars = None
    k=None
    iterations =100
    convergence_th=1e-3
    ric=None

    def __init__(self, n_components=1, tol=1e-3, max_iter=100):
        """
        A Gaussian mixture model trained via the expectation maximization
        algorithm.

        Parameters
        ----------

        n_components: The number of mixture components.
        tol: The convergence threshold
        """
        self.k=n_components
        self.iterations=max_iter
        self.convergence_th=tol


    def initialize_params(self, X):
        """
        Initialize the GMM parameters.

        Parameters
        ----------
        X : A collection of `N` training data points, each with dimension `d`.
        """

        self.means = np.random.rand(self.k,X.shape[1])
        self.covars = np.zeros((self.k, X.shape[1], X.shape[1]))
        for k in range(self.k):
            self.covars[k] = np.eye(X.shape[1])
        self.weights=[1/self.k]*self.k
        self.ric = np.zeros((X.shape[0], self.k))

    def E_step(self, X):
        """
        Find the Expectation of the log-likelihood evaluated using the current estimate for the parameters

        Parameters
        ----------
        X : A collection of `N` data points, each with dimension `d`.
        """
        n=X.shape[0]
        d = X.shape[1]

        nan_mask = np.isnan(self.covars)
        inf_mask = np.isinf(self.covars)
        self.covars[nan_mask] = 1e-6
        self.covars[inf_mask] = 1e-6

        for c in range(self.k):
            self.ric[:,c]= self.weights[c]*multivariate_normal.pdf(X,mean=self.means[c],cov=self.covars[c],allow_singular=True)
        for i in range(n):
            self.ric[i] = self.ric[i]/np.sum(self.ric[i])
        return

    def M_step(self, X):
        """
        Updates parameters maximizing the expected log-likelihood found on the E step.

        Parameters
        ----------
        X : A collection of `N` data points, each with dimension `d`.
        """
        n=X.shape[0]
        d=X.shape[1]

        for c in range(self.k):
            self.weights[c]=np.sum(self.ric[:,c])/n

        for c in range(self.k):
            clusterSum = np.sum(self.ric[:, c])
            self.means[c] = np.sum(X * self.ric[:, c][:, np.newaxis], axis=0) / clusterSum
            diff = X - self.means[c]

            self.covars[c] = (np.dot(self.ric[:, c] * diff.T, diff)/ clusterSum)
            self.weights[c] = clusterSum / n



    def fit(self, X, y=None):
        """
        Fit the parameters of the GMM on some training data.

        Parameters
        ----------
        X : A collection of `N` training data points, each with dimension `d`.
        y: not used
        """
        self.initialize_params(X)
        for i in range(self.iterations):
            self.E_step(X)
            self.M_step(X)

    def predict(self, X):
        """
        Predict the labels for the data samples in X using trained model.

        Parameters
        ----------
        X : A collection of `M` data points, each with dimension `d`.

        Returns
        -------
        Predicted labels.
        """
        N = X.shape[0]
        gamma = np.zeros((N, self.k))
        for k in range(self.k):
            pdf = multivariate_normal(mean=self.means[k], cov=self.covars[k])
            gamma[:, k] = self.weights[k] * pdf.pdf(X)
        return np.argmax(gamma, axis=1)

Let's start using the train images to fit the Gaussian mixture model. Note that this part could take a long time. (The code can be optimized by vectorization to avoid a nested for-loop)

In [12]:
from tqdm import tqdm
from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal

gmm_background = np.zeros(shape = (train_frames.shape[1], train_frames.shape[2], 3))

for i in tqdm(range(train_frames.shape[1])):
    for j in range(train_frames.shape[2]):
        X = train_frames[:, i, j, :]
        X = X.reshape(X.shape[0], 3)
        gmm = GMM(2,max_iter=5)
        gmm.fit(X)
        means = gmm.means
        weights = gmm.weights
        idx = np.argmax(weights)
        gmm_background[i][j][:] = means[idx]


  0%|          | 0/480 [00:00<?, ?it/s]/tmp/ipython-input-1747011717.py:69: RuntimeWarning: invalid value encountered in divide
  self.ric[i] = self.ric[i]/np.sum(self.ric[i])
  0%|          | 1/480 [00:03<30:17,  3.79s/it]/tmp/ipython-input-1747011717.py:88: RuntimeWarning: invalid value encountered in divide
  self.means[c] = np.sum(X * self.ric[:, c][:, np.newaxis], axis=0) / clusterSum
100%|██████████| 480/480 [28:23<00:00,  3.55s/it]


After a long time of training, we can now used the trained Gaussian mixture model to predict the foreground from the background given a seuqence of ordered images. The results looks failry nice.

However, the current implementation does not have a convergence check. It always runs for all iterations, so it is unclear how good the estimation is on the trianing data, or if it already converges but we still continue to train.

In [19]:
import cv2
import numpy as np
import imageio
import os
from IPython.display import HTML
from base64 import b64encode

# Assume test_frames and gmm_background are already defined
# test_frames: (N, H, W, 3) with float64 [0,1]
# gmm_background: (H, W, 3) with float64 [0,1]

# Define some parameters
threshold = 50  # intensity threshold for binary mask
fps = 10        # video playback speed

frames_out = []  # will hold stacked RGB frames

for i in range(test_frames.shape[0]):
    # Compute foreground difference (absolute diff between frame and background)
    diff = np.abs(test_frames[i]*255 - gmm_background*255)
    diff = diff.astype(np.uint8)

    # Convert to grayscale for thresholding
    foregrounds = cv2.cvtColor(diff, cv2.COLOR_RGB2GRAY)

    # Apply threshold (vectorized)
    _, binary = cv2.threshold(foregrounds, threshold, 255, cv2.THRESH_BINARY)

    # Convert single-channel binary mask → 3-channel RGB
    binary_rgb = cv2.cvtColor(binary, cv2.COLOR_GRAY2RGB)

    # Stack original + mask side-by-side
    stacked = np.hstack(( (test_frames[i]*255).astype(np.uint8), binary_rgb ))

    frames_out.append(stacked)

# Save as video
os.makedirs("output", exist_ok=True)
video_path = "output/foreground_extraction.mp4"
imageio.mimsave(video_path, frames_out, fps=fps)

# Display inline in Colab
mp4 = open(video_path, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=640 controls>
    <source src="{data_url}" type="video/mp4">
</video>
""")


/tmp/ipython-input-3825461699.py:21: RuntimeWarning: invalid value encountered in cast
  diff = diff.astype(np.uint8)


I found that OpenCV comes with a built-in function `cv2.createBackgroundSubtractorMOG2` which runs background subtraction very fast. However, it is interesting to observe that if the person falls and lies still, it is considered as part of the background.

In [20]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import imageio
from IPython.display import HTML
from base64 import b64encode

folder = "/content/fall_scenario8_folder/fall_scenario8_cam2/"
frame_paths = sorted(glob.glob(os.path.join(folder, "*.png")))

# --- Initialize the Background Subtractor ---
bg_subtractor = cv2.createBackgroundSubtractorMOG2(
    history=60,
    varThreshold=32,
    detectShadows=True
)

frames_out = []  # store concatenated frames for video output

for idx, path in enumerate(frame_paths):
    frame = cv2.imread(path)
    if frame is None:
        continue

    # Apply MOG2
    fg_mask = bg_subtractor.apply(frame)

    # Clean up small noise
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)

    # Convert mask to 3-channel grayscale for concatenation
    mask_rgb = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)

    # Concatenate original frame and mask side by side
    concat = cv2.hconcat([
        frame,
        mask_rgb
    ])

    # Resize for lighter video (optional)
    concat = cv2.resize(concat, (960, 480))

    # Convert BGR→RGB for correct color when saving
    frames_out.append(cv2.cvtColor(concat, cv2.COLOR_BGR2RGB))

# --- Save as MP4 video ---
os.makedirs("output", exist_ok=True)
video_path = "output/bg_subtraction.mp4"
imageio.mimsave(video_path, frames_out, fps=10)

# --- Display inline ---
mp4 = open(video_path, "rb").read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML(f"""
<video width=960 controls>
    <source src="{data_url}" type="video/mp4">
</video>
""")
